In [43]:
from settings import *

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [45]:
ppfeatures = [
    "vesselId", 
    "time",
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [46]:
input_features = [
    "vesselId", 
    "time",
    'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

output_features = [
    "vesselId", 
    "time",
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [47]:
ais_train = pd.read_csv(AIS_TRAIN, sep='|')
ais_train['time'] = pd.to_datetime(ais_train['time'])

In [48]:
# vessels = pd.read_csv(VESSELS, sep='|')
# vessels.head()

In [49]:
# pd.read_csv(SCHEDULES_TO_MAY_2024, sep="|")

In [50]:
ais_test = pd.read_csv(AIS_TEST, sep=",")
ais_test['time'] = pd.to_datetime(ais_test['time'])

In [51]:
# ais_test.head()

In [ ]:
len(ais_test["vesselId"].unique())

In [53]:

# def make_features(df: pd.DataFrame) -> pd.DataFrame:
#     raw = df.copy()

#     # time_diff: difference of time since last measure
#     raw['time'] = pd.to_datetime(raw['time'])
#     raw['time_diff'] = raw.sort_values(by=['vesselId', 'time']).groupby("vesselId")['time'].diff()
    
    
#     return raw

In [54]:
# ais_tr = make_features(ais_train)

In [55]:
# te.sort_values(by=["vesselId", "time"])

In [ ]:
ais_train["portId"].isna().sum()

In [ ]:
ais_train

In [58]:
# CREATE time_diff AND MAKE IT IN SECONDS

train_vessel_id_time = ais_train[ppfeatures].copy()
train_vessel_id_time["split"] = "train"
train_vessel_id_time["ID"] = train_vessel_id_time.index

test_vessel_id_time = ais_test[["ID", "vesselId", "time" ]].copy()
test_vessel_id_time["split"] = "test"
ais_data = pd.concat([train_vessel_id_time, test_vessel_id_time], ignore_index=True)

# arrival time diff (from etaRaw)
# 
ais_data['time_diff'] = (
    ais_data
    .sort_values(by=['time'])
    .groupby("vesselId")['time']
    .diff()
    .dropna()
    .dt.total_seconds()
    .astype(int)
    .shift(-1)
)


def update_split_column(group: pd.Series) -> pd.Series:
    if group.name in ais_test["vesselId"].unique():
        group.iloc[-1] = "both"
    return group

df_temp = ais_data.copy()

ser_temp = pd.Series(
    df_temp[df_temp["split"]=="train"]
    .sort_values(by="time")
    .groupby("vesselId")["split"]
    .apply(update_split_column)
    .reset_index(drop=True)
)
ser_temp.index = df_temp[df_temp["split"]=="train"].sort_values(["vesselId", "time"]).index

ais_data.loc[ser_temp.index, "split"] = ser_temp

In [ ]:
ais_data.describe()

In [60]:
data_train = ais_data[(ais_data["split"]=="train")|(ais_data["split"]=="both")]
data_test = ais_data[(ais_data["split"]=="test")|(ais_data["split"]=="both")]

In [61]:
scaled_features = [
    'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]
scaled_features_y = [
    # 'time_diff',
    'cog',
    'sog',
    'rot',
    'heading',
    # 'navstat',
    # 'etaRaw',
    'latitude',
    'longitude',
]

In [62]:
scaler = StandardScaler()
scaler_y = StandardScaler()

In [63]:
data_train = data_train.dropna(subset="time_diff")

In [64]:
X = data_train.sort_values(["vesselId", "time"]).iloc[:-1][input_features]
y = data_train.sort_values(["vesselId", "time"]).iloc[1:][output_features]

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=False)

In [66]:
X_train_scaled = X_train.copy()
X_train_scaled[scaled_features] = scaler.fit_transform(X_train_scaled[scaled_features])

y_train_scaled = y_train.copy()
y_train_scaled[scaled_features_y] = scaler_y.fit_transform(y_train_scaled[scaled_features_y])

In [67]:
X_test_scaled = X_test.copy()
X_test_scaled[scaled_features] = scaler.transform(X_test_scaled[scaled_features])

y_test_scaled = y_test.copy()
y_test_scaled[scaled_features_y] = scaler_y.transform(y_test_scaled[scaled_features_y])

In [68]:
# X_train = X_train.sort_values(["time", "vesselId"])
# y_train = y_train.sort_values(["time", "vesselId"])
# X_test = X_test.sort_values(["time", "vesselId"])
# y_test = y_test.sort_values(["time", "vesselId"])

In [69]:
import xgboost as xgb

from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV

In [70]:
params = {
    'n_estimators': 5000,
    'gamma': 0.5,
    'subsample': 0.6,
    'n_estimators': 5000,
    'min_child_weight':  15,
    'colsample_bytree': 0.8,
    'max_depth': 4,
    'eta':  0.005,
    'refresh_leaf': 1,
}

In [71]:
param_grid = {
    'n_estimators': [5000, 10_000],
    'gamma': [0.5,],
    'subsample': [0.6,],
    'min_child_weight':  [15, 20],
    'colsample_bytree': [0.8,],
    'max_depth': [4, 10],
    'eta':  [0.005,],
    'refresh_leaf': [1,],

}

In [72]:
X_train = X_train_scaled[scaled_features]
y_train = y_train_scaled[scaled_features_y]
X_test = X_test_scaled[scaled_features]
y_test = y_test_scaled[scaled_features_y]

In [73]:
xgb_model = xgb.XGBRegressor(**params)

# grid_search = GridSearchCV(xgb_model, param_grid=param_grid)
# grid_search.fit(
#     X_train, y_train,
#     eval_set=[(X_train, y_train), (X_test, y_test)]
# )
# xgb_model = grid_search.best_estimator_
# print(grid_search.best_params_)

In [ ]:
xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    # early_stopping_rounds=50,
    verbose=False,
)
# xgb_model.load_model("models/xgb_model_v2.json")

In [75]:
try:
    xgb_model.save_model("models/xgb_model_v2_1.json")
except:
    xgb_model.save_model("models/xgb_model_v2_2.json")

In [76]:
# import matplotlib.pyplot as plt

# results = xgb_model.evals_result()
# train_errors = results['validation_0']['rmse']
# test_errors = results['validation_1']['rmse']

# plt.figure(figsize=(10, 6))
# plt.plot(train_errors, label='Train')
# plt.plot(test_errors, label='Test')
# plt.xlabel('Boosting Rounds')
# plt.ylabel("rmse")
# plt.legend()
# plt.title('Learning Curves')
# plt.show()

In [77]:
y_pred = xgb_model.predict(X_test)

In [ ]:
len_test = len(y_test)
len_test

In [79]:
df_y = y.iloc[-len_test:]

In [ ]:
y_test

In [ ]:
y_test.columns

## Visualize

In [ ]:
df_y = y.iloc[-len_test:].copy()
df_y = df_y.reset_index()
vessel_id = df_y["vesselId"].unique()[104]
print("vesselId:", vessel_id)
index = list(df_y[df_y["vesselId"]==vessel_id].index)

y_test_plot = y_test.reset_index().iloc[index]
y_pred_plot = pd.DataFrame(y_pred.take(index, axis=0), columns=y_test.columns)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(y_test_plot["longitude"], y_test_plot["latitude"], 'o', label="test")
plt.plot(y_pred_plot["longitude"], y_pred_plot["latitude"], 'o', label="pred")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(y_test_plot["longitude"], y_test_plot["latitude"], label="test")
plt.plot(y_pred_plot["longitude"], y_pred_plot["latitude"], label="pred")
plt.legend()

In [86]:

# vessel_id = y_test.values

In [87]:
# from src.eval.metric import *

# score(y_test[["ID", "latitude", "longitude"]], y_pred, "ID")

## Forecast

In [195]:

def iterative_forecast(last_known, model, steps, sequence_length):
    predicted = []
    current_sequence = last_known[0].reshape(1,-1)
    # current_sequence = last_known[-sequence_length:]
    for k in range(steps):
        # next_pred = model.predict(current_sequence.reshape(1, sequence_length, -1))[0]
        next_pred = model.predict(current_sequence)[0]
        # TODO: use ais_test["time_diff"] (not predicted currently)
        predicted.append(next_pred)
        last_known[k+1,1:] = next_pred
        # Update current_sequence by appending next prediction
        current_sequence = last_known[k+1].reshape(1,-1)
    
    return predicted

In [196]:
grouped_test = data_test.groupby("vesselId")

In [ ]:
predictions = []
forecast_steps = 300
sequence_length = 1

for vessel_id, group in grouped_test:
    forecast_steps = len(group['time'].values) - 1

    # last_known_features = scaler.transform(group[scaled_features].values[-sequence_length:])
    last_known_features = scaler.transform(group[scaled_features].values)
    future_preds = iterative_forecast(last_known_features, xgb_model, forecast_steps, sequence_length)
    
    # Store the predictions
    df_pred = pd.DataFrame(scaler_y.inverse_transform(future_preds), columns=scaled_features_y)
    df_pred['time'] = group['time'].iloc[1:].values
    df_pred["vesselId"] = vessel_id
    predictions.append(df_pred)


In [ ]:
df_pred.describe()

In [ ]:
group[scaled_features].values[-sequence_length:]

In [ ]:
last_known_features

In [202]:
df_preds = pd.concat(predictions, ignore_index=True)

In [ ]:
df_preds.describe()

In [203]:
res = pd.merge(ais_test, df_preds[["vesselId","time", "latitude", "longitude"]],on=["time", "vesselId"], how="left")

In [204]:
res["longitude_predicted"] = res["longitude"]
res["latitude_predicted"] = res["latitude"]
# res["id"] = res["ID"]
res = res.drop(columns=["longitude", "latitude"])

In [ ]:
res[["ID","longitude_predicted","latitude_predicted"]]

In [ ]:
from utils import *

from settings import *

from datetime import datetime
import uuid

import pandas as pd

def make_file_name() -> str:
    file_name = str(uuid.uuid4()) + ".csv"
    print(f"Submission file name is: {file_name}")
    return file_name

def submit(forecast: pd.DataFrame, file_name: str = None) -> None:
    sample_submission = pd.read_csv(AIS_SAMPLE_SUBMISSION)
    file_name = file_name if file_name else make_file_name()

    repertory = SUBMISSION_FODLER.joinpath(file_name)
    sample_submission = sample_submission[['ID']].merge(forecast[["ID","longitude_predicted","latitude_predicted"]], on='ID', how='left')
    try:
        sample_submission.to_csv(repertory, index=False)
    except:
        print("Error register file")
        submit(forecast)

submit(res)

In [ ]:
res["longitude_predicted"].plot()

In [ ]:
y_test["longitude"].plot()